# Implementation of Theorem 18 for the quadratic form $x^2 + (2z+1)y^2 = 128n(2z+1)^2$

# Library Importation

In [1]:
import sympy as sp
import pandas as pd
from IPython.display import display

# Class definitions and solution construction

## Diophantine Solver for $x^2 + (2z+1)y^2 = 128n(2z+1)^2$ 

In [2]:
class DiophantineSolver:
    def __init__(self, z: int, max_n: int = 50, max_solutions_per_n: int = 10):
        self.z = z
        self.max_n = max_n
        self.max_solutions_per_n = max_solutions_per_n
        self.x, self.y = sp.symbols('x y', integer=True)
        self.a = 2 * z + 1
        self.mod_val = 16 * self.a
        self.constraint = 8 * self.a
        self.results_df = pd.DataFrame(columns=["x", "y", "n"])

    def _satisfies_constraints(self, x_val, y_val):
        return (
            x_val >= 0 and
            y_val >= 0 and
            x_val % self.mod_val == self.constraint and
            y_val % self.mod_val == self.constraint
        )

    def _solve_for_n(self, n):
        rhs = 128 * n * self.a**2
        eq = sp.Eq(self.x**2 + self.a * self.y**2, rhs)
        try:
            solutions = sp.diophantine(eq)
            count = 0
            seen = set()

            for sol in solutions:
                if len(sol) != 2:
                    continue
                x_expr, y_expr = sol
                try:
                    x_val = int(x_expr.evalf())
                    y_val = int(y_expr.evalf())
                except (TypeError, ValueError):
                    continue

                if self._satisfies_constraints(x_val, y_val) and (x_val, y_val) not in seen:
                    self.results_df.loc[len(self.results_df)] = [x_val, y_val, n]
                    seen.add((x_val, y_val))
                    count += 1

                if count >= self.max_solutions_per_n:
                    break

        except Exception as e:
            print(f"Error solving for n = {n}: {e}")

    def solve(self):
        for n in range(1, self.max_n + 1):
            self._solve_for_n(n)
        return self.results_df

## Pell solver for $a^2 - 2b^2 = 2z+1$ and $u^2 - 2v^2 = -2$

In [3]:
class PellSolver:
    def __init__(self, z: int):
        self.z = z
        self.A, self.B = sp.symbols('a b', integer=True)
        self.U, self.V = sp.symbols('u v', integer=True)
        self.results = {}

    def _extract_sample_solution(self, solns):
        for sol in solns:
            if all(expr.free_symbols == set() for expr in sol):
                return tuple(abs(int(expr)) for expr in sol)
            else:
                params = set()
                for expr in sol:
                    params.update(expr.free_symbols)
                subs = {param: 1 for param in params}
                try:
                    return tuple(abs(int(expr.subs(subs).evalf())) for expr in sol)
                except Exception:
                    continue
        return None

    def solve_pell_equation_1(self):
        rhs = 2 * self.z + 1
        eq1 = sp.Eq(self.A**2 - 2 * self.B**2, rhs)
        solns1 = sp.diophantine(eq1)
        self.results['eq1'] = self._extract_sample_solution(solns1)

    def solve_pell_equation_2(self):
        eq2 = sp.Eq(self.U**2 - 2 * self.V**2, -2)
        solns2 = sp.diophantine(eq2)
        self.results['eq2'] = self._extract_sample_solution(solns2)

    def solve(self):
        self.solve_pell_equation_1()
        self.solve_pell_equation_2()
        return self.results

## Solution triple construction w.r.t Theorem 18

In [4]:
def generate_triples_extended(dio_solver, pell_sol1, pell_sol2, iterations=5):
    a, b = pell_sol1
    u0, v0 = pell_sol2
    u, v = u0, v0

    # Transformation matrix T
    T = sp.Matrix([[3, 4], [2, 3]])
    vec = sp.Matrix([u, v])

    all_triples = pd.DataFrame()

    for k in range(iterations):
        label = f"({vec[0]},{vec[1]})"

        triples = []

        for idx, row in dio_solver.results_df.iterrows():
            x, y, n = row['x'], row['y'], row['n']

            denom = 16 * (2 * dio_solver.z + 1)
            numer_const = 8 * (2 * dio_solver.z + 1)

            x_k = (x - numer_const) / denom
            y_k = (y - numer_const) / denom

            if not (x_k.is_integer() and y_k.is_integer()):
                continue

            x_k, y_k = int(x_k), int(y_k)

            term1 = (2 * y_k + 1) * (b * vec[0] + a * vec[1])
            term2 = (2 * x_k + 1)

            X = (term1 + term2) // 2
            Y = (term1 - term2) // 2
            Z = ((2 * y_k + 1) * (a * vec[0] + 2 * b * vec[1])) // 2

            triples.append({'x': x, 'y': y, 'n': n, label: (X, Y, Z)})

        if triples:
            triples_df = pd.DataFrame(triples)
            if all_triples.empty:
                all_triples = triples_df
            else:
                all_triples = pd.merge(all_triples, triples_df, on=['x', 'y', 'n'], how='outer')

        vec = T @ vec

    return all_triples



# User input, initlization, and dataframe display

In [5]:
def run_full_solver():
    z = int(input("Enter z: "))
    max_n = int(input("Enter max_n: "))
    max_solutions_per_n = int(input("Enter max_solutions_per_n: "))
    iterations = int(input("Enter number of iterations for matrix multiplication: "))

    dio_solver = DiophantineSolver(z=z, max_n=max_n, max_solutions_per_n=max_solutions_per_n)
    dio_solver.solve()

    pell_solver = PellSolver(z=z)
    pell_results = pell_solver.solve()

    if not pell_results.get('eq1') or not pell_results.get('eq2'):
        print("No Pell solutions found.")
        return pd.DataFrame()

    a, b = map(abs, pell_results['eq1'])
    u, v = map(abs, pell_results['eq2'])

    T = sp.Matrix([[3, 4], [2, 3]])
    uv_vec = sp.Matrix([[u], [v]])

    for _ in range(iterations):
        u_n, v_n = int(uv_vec[0]), int(uv_vec[1])
        triples = []

        for _, row in dio_solver.results_df.iterrows():
            x, y, n = row['x'], row['y'], row['n']
            denom = 16 * (2 * z + 1)
            numer_const = 8 * (2 * z + 1)

            x_k = (x - numer_const) / denom
            y_k = (y - numer_const) / denom

            if not (x_k.is_integer() and y_k.is_integer()):
                triples.append(None)
                continue

            x_k = int(x_k)
            y_k = int(y_k)

            term1 = (2 * y_k + 1) * (b * u_n + a * v_n)
            term2 = (2 * x_k + 1)

            X = (term1 + term2) // 2
            Y = (term1 - term2) // 2
            Z = ((2 * y_k + 1) * (a * u_n + 2 * b * v_n)) // 2

            triples.append((X, Y, Z))

        col_name = f"({u_n},{v_n})"
        dio_solver.results_df[col_name] = triples
        uv_vec = T @ uv_vec

    return dio_solver.results_df

## Run Output

In [6]:
final_df = run_full_solver()
display(final_df)  # Use display() if in a Jupyter environment, or print(final_df) otherwise

,x,y,n,"(4,3)","(24,17)","(140,99)","(816,577)","(4756,3363)"
0,56,56,4,"(38, 37, 53)","(219, 218, 309)","(1274, 1273, 1801)","(7423, 7422, 10497)","(43262, 43261, 61181)"
1,168,56,8,"(39, 36, 53)","(220, 217, 309)","(1275, 1272, 1801)","(7424, 7421, 10497)","(43263, 43260, 61181)"
2,280,56,16,"(40, 35, 53)","(221, 216, 309)","(1276, 1271, 1801)","(7425, 7420, 10497)","(43264, 43259, 61181)"
3,392,56,28,"(41, 34, 53)","(222, 215, 309)","(1277, 1270, 1801)","(7426, 7419, 10497)","(43265, 43258, 61181)"
4,56,168,32,"(113, 112, 159)","(656, 655, 927)","(3821, 3820, 5403)","(22268, 22267, 31491)","(129785, 129784, 183543)"
5,168,168,36,"(114, 111, 159)","(657, 654, 927)","(3822, 3819, 5403)","(22269, 22266, 31491)","(129786, 129783, 183543)"
6,504,56,44,"(42, 33, 53)","(223, 214, 309)","(1278, 1269, 1801)","(7427, 7418, 10497)","(43266, 43257, 61181)"
7,392,168,56,"(116, 109, 159)","(659, 652, 927)","(3824, 3817, 5403)","(22271, 22264, 31491)","(129788, 129781, 183543)"
8,616,56,64,"(43, 32, 53)","(224, 213, 309)","(1279, 1268, 1801)","(7428, 7417, 10497)","(43267, 43256, 61181)"
9,504,168,72,"(117, 108, 159)","(660, 651, 927)","(3825, 3816, 5403)","(22272, 22263, 31491)","(129789, 129780, 183543)"
